# Introduction

Writing effective algorithms with Python requires comptence with the <span style="font-family:'Courier New'">numpy</span> package, which enables:
- Fast execution
- Minimum memory footprint

Understanding <span style="font-family:'Courier New'">numpy</span>, its speed, and how to use it more effectively requires a deeper examination of how variables are handled in memory.    

This Jupyter notebook covers the essential basis of <span style="font-family:'Courier New'">numpy</span> syntax and methods to get you on your way to writing faster code that uses less memory.  It focuses on basic techniques that are frequently useful in writing algorithms.

# <span style="font-family:'Courier New'">numpy</span> versus Python Lists

The first things to understand is that <code>numpy</code> derives its speed from storing elements of arrays in contiguous blocks of memory and relying on the fast C programming language.

![contiguous_memory](images/numpy_vs_list.jpg)

# Speed: Establish <span style="font-family:'Courier New'">numpy</span> Variables Once <a class="anchor" id="instan_numpy-once">

Any of these actions cause a <code>numpy</code> array to be re-instantiated, which negates the speed advantages you could garner with <code>numpy</code>.  So, avoid these operations:
    
- Resize a <span style="font-family:'Courier New'">numpy</span> array 
- Change the data type of a <span style="font-family:'Courier New'">numpy</span> array 
- Concatenating <span style="font-family:'Courier New'">numpy</span> arrays, <code>np.concatenate()</code>
- Appending values to <span style="font-family:'Courier New'">numpy</span> arrays, <code>np.append()</code>
- Using <code>numpy np.vstack()</code> or <code>np.hstack()</code>

Prevent needing to re-instantiating a <span style="font-family:'Courier New'">numpy ndarray</span>s by determining the required _size_, _shape_, and _data type_ of the array and establish it once.

Assume in the example below we are are filling a <span style="font-family:'Courier New'">numpy</span> array with values that we are computing, which I will simulate with random numbers.  We will do this two ways, one with reserving space for the results first, and, the second, appending rows to the original <span style="font-family:'Courier New'">numpy</span> array as we create the values.  The second way causes the numpy array to be reconstructed many times.

In [1]:
import numpy as np
import time

In [2]:
nrows = 10000
ncols = 10

The numpy methods used below recreate the <code>numpy</code> array each time the method is called, which caused execution to be incredibly slow.

- <code>np.hstack()</code>
- <code>np.vstack()</code>
- <code>np.concatenate()</code>
- <code>np.append()</code>

These methods applied multiple times within a loop is a poor idea: find a better, faster way.

Operations with <code>numpy</code> arrays are very fast once the arrays are created, but instantiating a <code>numpy</code> array takes non-neglible time.

In [4]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.vstack((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.13271784782409668
(10000, 10)


In [12]:
start = time.time()
np_arr = np.random.rand(ncols,1)
for i in range(nrows-1):
    np_arr = np.hstack((np_arr, np.random.rand(ncols,1)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.14214706420898438
(10, 10000)


In [5]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.append(np_arr, np.random.rand(1,ncols), axis=0)
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.12223935127258301
(10000, 10)


In [6]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.concatenate((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.11562895774841309
(10000, 10)


It is much faster to create a numpy array of sufficient size once to reserve space and just replace its values as the algorithm progresses.

In [7]:
np_res = np.zeros((nrows, ncols))

In [8]:
start = time.time()
for i in range(nrows):
    new_row = np.random.rand(1,ncols)
    np_res[i] = new_row
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_res.shape)

Execution time with reserved ndarray: 0.015741586685180664
(10000, 10)


If you do feel the need to append to a data structure as an algorithm progresses without defining a <code>numpy</code> array to reserve space, then accumulating data initially in a Python list before converting to a <code>numpy</code> is much faster.

In [10]:
start = time.time()
np_arr = []
for i in range(nrows-1):
    np_arr.append(np.random.rand(1,ncols)) 
print(f'Execution time with reserved ndarray: {time.time() - start}')
np_arr = np.array(np_arr)
print(np_arr.shape)

Execution time with reserved ndarray: 0.013594865798950195
(9999, 1, 10)


## Application: The Cell Tower Problem

In [3]:
import random

def setup():
    prob_size = 100000
    data = [random.random() for _ in range(prob_size)]
    budget = 5.0
    return data, budget

### Python List with Deletion of Used Elements

### With a <code>for</code> Loop

In [4]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

for t in towers:
    if sum(towers_to_pick) + t <= budget:
        towers_to_pick.append(t)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 4.99999431628723 
Execution time: 0.03404688835144043 seconds 
Towers selected: [0.9260635362140286, 0.14366894613766779, 0.07432638083616072, 0.4166398903870302, 0.9485426719217637, 0.48905463043213826, 0.8287739234592457, 0.08041245002898012, 0.6144388138240645, 0.26195785910302427, 0.04827605338732366, 0.13517624811127593, 0.01344953589619613, 0.010675630305102546, 0.0028299071892031735, 0.0006726011731652548, 0.001085913921595072, 0.0035709381623647563, 0.0003390969368196606, 1.7875562511204812e-05, 2.1413297568440015e-05]


### With a <code>while</code> Loop

In [5]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers.pop(0))
    else:
        _ = towers.pop(0)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 4.9999989170564785 
Execution time: 0.661456823348999 seconds 
Towers selected: [0.8106410485553919, 0.7174504660364858, 0.442659594792551, 0.3230989743545274, 0.11711954562943805, 0.04246052614156792, 0.5199552347896623, 0.9445431073541483, 0.5824113193036726, 0.018972549648881287, 0.09401756683524376, 0.04041010848036641, 0.051436703513252136, 0.06571415314078821, 0.16717186926298422, 0.04693154903099961, 0.007595579818323039, 0.0036825263151525345, 0.002073985632695452, 0.00022921270903497604, 0.0006071861037938442, 0.00012758207669538724, 0.0002485221277603866, 0.00015165205476252996, 9.797820453483208e-05, 0.0001401887767300547, 5.018636703357782e-05]


In [6]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []
while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers[0])
    del towers[0]
    # towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 4.999989575574036 
Execution time: 0.7659039497375488 seconds 
Towers selected: [0.438637854801326, 0.2971294162002962, 0.9072132834178651, 0.35362741546089416, 0.28563054096532514, 0.4926932611022279, 0.6908594861948345, 0.4748780114060528, 0.4565977391553562, 0.23323356722756694, 0.0018972935500413124, 0.17624885880123742, 0.06217725399262053, 0.09856985332531754, 0.020353864010952738, 0.009985236902834593, 0.00017254594290494474, 6.731856241948275e-05, 1.677455396142946e-05]


### <code>numpy</code> with Slices to Delete Used Elements

In [7]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

towers_to_pick = np.array([])

while towers_to_pick.sum() < budget and towers.shape[0] > 0:
    if towers_to_pick.sum() + towers[0] <= budget:
        towers_to_pick = np.append(towers_to_pick, towers[0])
    towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 4.9999840193253915 
Execution time: 0.3738241195678711 seconds 
Towers selected: [9.47046431e-02 9.83881163e-01 3.79981606e-01 2.61839381e-01
 5.18176966e-01 4.42555483e-01 5.49239293e-01 9.48243547e-01
 3.37406149e-01 2.96430802e-01 5.24496720e-02 3.53301547e-03
 7.59467365e-02 2.43019146e-02 3.08101958e-02 9.05554179e-05
 1.06461143e-05 1.08015796e-04 1.32658701e-05 1.40742587e-04
 5.04534752e-06 5.78288033e-05 4.09460918e-05 1.37176687e-05
 2.68746393e-06]


### Efficient <code>numpy</code> with Reserved Memory for Array

Notice also that the <code>numpy</code> array <code>sum()</code> function is replaced.

In [8]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

''' Reserve space for solution of maximum possible size '''
towers_to_pick = np.zeros(towers.shape[0], dtype=np.float32)  # do not use np.empty()!!!

j = 0  # counter for number of elements packed and the index of the next element to be packed
for vol in towers:
    if vol <= budget:
        towers_to_pick[j] = vol
        budget -= vol
        j += 1

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick[:j]}')

Investment: 4.999977597165042 
Execution time: 0.01894855499267578 seconds 
Towers selected: [6.7614907e-01 7.2852272e-01 1.9289312e-01 2.7876312e-01 9.0021110e-01
 2.9800627e-01 9.4279552e-01 7.4269390e-01 5.6808151e-02 1.2176196e-01
 5.5007875e-02 5.0451737e-03 1.2046301e-03 1.0442288e-04 5.0826120e-06
 5.3700592e-06 1.2918187e-07]


## Avoid Loops with <code>numpy</code> and Elementwise Calculations (Vectorization)

As we have discussed, your code slows dramatically with each nested <code>for</code> loop you add.  You can avoid using loops with <code>numpy</code> vectorization.  While the loops are eliminated from your Python code, a looping mechanism is still executed behind the scenes in <code>numpy</code>, although <code>numpy</code> does this operation much more quickly than if it was done with Python code.

### The Traditional Python Approach to Array Addition with Loops

In [ ]:
x = [[0,1,2],[3,4,5],[6,7,8]]
y = [[1,1,1],[1,1,1],[1,1,1]]
z = [[0,0,0],[0,0,0],[0,0,0]]

for i in range(len(x)):
    for j in range(len(x[0])):
        z[i][j] = x[i][j] + y[i][j]
print(z)

### Array Addition With <code>numpy</code>, Without loops

In [ ]:
x = np.array([[0,1,2],[3,4,5],[6,7,8]])
y = np.array([[1,1,1],[1,1,1],[1,1,1]])

z = x + y
print(z)

### A Bigger Addition Problem

In [17]:
import random
prob_size = 1000
x = [[random.randint(0,10) for _ in range(prob_size)] for _ in range(prob_size)]
y = [[random.randint(0,10) for _ in range(prob_size)] for _ in range(prob_size)]
z = [[0 for _ in range(prob_size)] for _ in range(prob_size)]

time_start = time.time()
for i in range(len(x)):
    for j in range(len(x[0])):
        z[i][j] = x[i][j] + y[i][j]
print(f'for loop execution time: {time.time() - time_start}')

x = np.random.randint((prob_size,prob_size))
y = np.random.randint((prob_size,prob_size))

time_start = time.time()
z = x + y
print(f'numpy execution time: {time.time() - time_start}')

for loop execution time: 0.2594606876373291
numpy execution time: 0.0


## Selecting Elements from <code>numpy</code> Arrays 

Before we continue with the topic of writing faster code, let's refresh or learn about some very useful <code>numpy</code> methods.

- <span style="font-family:'Courier New'">np.min()</span>
- <span style="font-family:'Courier New'">np.max()</span>
- <span style="font-family:'Courier New'">np.argmin()</span>
- <span style="font-family:'Courier New'">np.argmax()</span>

Algorithms frequently require that either the minimum or maximum elements be selected from an array/list or, in a more complex manner, the best element fitting particular criteria is sought.

One one just find the least or greatest array elements using the <code>np.min()</code> or <code>np.max()</code> methods, respectively.

In [ ]:
x = np.random.randint(0,10,(10,))

In [18]:
print(x)
print(x.min(), np.min(x))
print(x.max(), np.max(x))

[9 7 5 1 9 8 3 9 0 9]
0 0
9 9


One might also find the leat and greatest elements using the <code>np.argmin()</code> or <code>np.argmax()</code> methods, respectively, although this requires a second statement to actually retrieve the element values.

In [19]:
idx_min = x.argmin()
idx_max = x.argmax()
print(x)
print(idx_min, x[idx_min])
print(idx_max, x[idx_max])

[9 7 5 1 9 8 3 9 0 9]
8 0
0 9


Despite needing a second statement to obtain a value, knowing the index of a minimum/maximum is quite useful when one must select multiple elements from an array and keep track of which elements have been selected so that they are not selected again.  This is the focus of a subsequent section in this Jupyter notebook.

The <code>np.argsort()</code> method can be useful to find the element from a list that, rather than being the least or greatest element, is the largest (smallest) item smaller (larger) than some upper (lower)limit.

In [27]:
idx_sort = np.argsort(x)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

x: [9 7 5 1 9 8 3 9 0 9]
idx_sort: [8 3 6 2 1 5 0 4 7 9]
x[idx_sort]: [0 1 3 5 7 8 9 9 9 9]


In [25]:
# Find the largest element less than 5
i = -1
while x[idx_sort[i+1]] < 5 and i+1 < x.shape[0]:
    i += 1
print(i, idx_sort[i], x[idx_sort[i]])

2 6 3


Recall one method for sorting in descending order.

In [28]:
idx_sort = np.argsort(x)
idx_sort = np.flip(idx_sort)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

x: [9 7 5 1 9 8 3 9 0 9]
idx_sort: [9 7 4 0 5 1 2 6 3 8]
x[idx_sort]: [9 9 9 9 8 7 5 3 1 0]


This is another method, although it is perhaps less intuitive.

In [29]:
idx_sort = np.argsort(x)
idx_sort = idx_sort[::-1]
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

x: [9 7 5 1 9 8 3 9 0 9]
idx_sort: [9 7 4 0 5 1 2 6 3 8]
x[idx_sort]: [9 9 9 9 8 7 5 3 1 0]


## Boolean Masks

A Boolean (<code>True</code>/<code>False</code>) array can be used to filter out values from a <code>numpy</code> array.  Array elements whose position coincide with a <code>False</code> are filtered out.

### Example 1

In [ ]:
size = 5
x = np.arange(size)
x

In [ ]:
mask_x = np.array([True if i%2==1 else False for i in range(size)])
mask_x

In [ ]:
print(x[mask_x])

### Example 2

In [ ]:
y = np.arange(size**2).reshape(5,5)
y

In [ ]:
mask_y = np.array([True if i%2==1 else False for i in range(size**2)]).reshape(5,5)
mask_y

In [ ]:
print(y[mask_y])

### Example 3

In [ ]:
z = np.random.random(size = (10,))
z

In [ ]:
z >= 0.5

In [ ]:
mask_z = (z >= 0.5)
mask_z

In [ ]:
z[mask_z]

## Application 2: Traveling Salesperson Problem

In this problem, the task is to maintain the original data in its original instantiation without deleting the data pertaining to the destinations already included in the Traveling Salesperson's route.

### Recall <code>np.argmin()</code>

Gets the index (argument) of the minimum value.

In [ ]:
z = np.random.random(10)
print(z)

In [ ]:
print(f'argmin(z) = {np.argmin(z)}; minimum value = {z[np.argmin(z)]}')

### A TSP Greedy Algorithm

Randomly select Location 1 to start.

- Loop until all locations visited
  - For each location, choose the next location to be closest possible next location of locations not yet visited
  
![AlgoStep1](images/m1.jpg)
![AlgoStep2](images/m2.jpg)
![AlgoStep3](images/m3.jpg)
![AlgoStep4](images/m4.jpg)
![AlgoStep5](images/m5.jpg)

Route: 1-2-0-4-3-1

#### Set up the data

In [ ]:
# create distance matrix
nloc = 10
dist = np.random.rand(nloc,nloc)
dist = np.triu(dist,k=0)
for i in range(1,nloc):
    for j in range(0, i):
        dist[i,j] = dist[j,i]
for i in range(nloc):
    dist[i,i]=0.0
dist

In [ ]:
''' Set up parameters '''
nloc = dist.shape[0]                      # number of locations
assert dist.shape[0] == dist.shape[1]     # ensure square distance matrix

''' Initialize random starting point '''
start = np.random.randint(0, nloc-1)      # select random starting location
sol = [start]                             # solution route in a list
cur_loc = start                           # use cur_loc to indicate current location index

''' Establish Boolean mask for the columns: True = column location not visited '''
col_mask = np.ones(nloc).astype(np.bool_) # creates array of True
col_mask[start] = False                   # cannot choose starting location as
                                          # next location

''' Create ndarray of column indices '''
col_indices = np.arange(nloc)             # create array of indices

''' Initial distance of solution '''
sol_dist = 0.0                            # initialize distance of solution

''' Execute algorithm '''
while col_mask.sum() > 0:              # continue if any True values in col_mask
    ''' Get index of next location '''
    next_loc_ind = np.argmin(dist[cur_loc][col_mask])  # get index of row minimum for
                                                       #  remaining locations
    next_loc_ind = col_indices[col_mask][next_loc_ind] # find index of minimum relative to original
                                                       #   indices (true index of location)
    
    ''' Update solution and mask '''
    sol.append(next_loc_ind)                   # append next location to solution
    col_mask[next_loc_ind] = False             # update mask for current location
    sol_dist += dist[cur_loc, next_loc_ind]    # update solution distance
    cur_loc = next_loc_ind                     # update current location

sol.append(start)       # append starting location for round trip
sol, sol_dist